# Create PySpark DataFrame

In [1]:
import os
import sys
import gc

---
# HDFS permission

For a non-spark user to be able to submit a job, login to the HDFS node as the hadoop user to run:

```
hadoop fs -mkdir /user/${USERNAME}
hadoop fs -chown ${USERNAME} /user/${USERNAME}
hadoop fs -chmod g+w /user/${USERNAME}
```

Otherwise an error:
```
21/08/15 21:15:28 ERROR SparkContext: Error initializing SparkContext.
org.apache.hadoop.security.AccessControlException: Permission denied: user=${USERNAME}, access=WRITE, inode="/user":hadoop:hadoop:drwxrwxr-x
```

---
# Environment variables

## HADOOP_CONF_DIR

Copy the **HADOOP_CONF_DIR** from the Hadoop/YARN master node and set the ```HADOOP_CONF_DIR``` environment variable locally to point to the directory.

* [Launching Spark on YARN
](http://spark.apache.org/docs/latest/running-on-yarn.html#launching-spark-on-yarn)

> Ensure that **HADOOP_CONF_DIR** or **YARN_CONF_DIR** points to the directory which contains the (client side) configuration files for the Hadoop cluster. These configs are used to write to HDFS and connect to the YARN ResourceManager. The configuration contained in this directory will be distributed to the YARN cluster so that all containers used by the application use the same configuration. 

In [2]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [3]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

---
# Spark Session


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

21/09/15 15:21:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/15 15:21:50 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# PySpark DataFrame

* [SparkSession.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html)

> Parameters
> * data: RDD or iterable
an RDD of any kind of SQL data representation(e.g. Row, tuple, int, boolean, etc.), or list, or pandas.DataFrame.
> * schema: pyspark.sql.types.DataType, str or list, optional

## From (data=List[Tuple], schema=List[str])

In [10]:
df = spark.createDataFrame(
    data=[
        ("messi", 1), 
        ("ronald", 2), 
        ("messi", 3), 
        ("ronald", 4)
    ], 
    schema=[
        "name", "goal"
    ]
)
df.show()

+------+----+
|  name|goal|
+------+----+
| messi|   1|
|ronald|   2|
| messi|   3|
|ronald|   4|
+------+----+



## From (data=List[Dictioary])

The dictionary key is the column name.

In [12]:
df = spark.createDataFrame(
    data=[
        {"name": "messi", "goal": 1}, 
        {"name": "ronald", "goal": 2}, 
        {"name": "messi", "goal": 1}, 
        {"name": "ronald", "goal": 4}
    ]
)
df.show()

+----+------+
|goal|  name|
+----+------+
|   1| messi|
|   2|ronald|
|   1| messi|
|   4|ronald|
+----+------+



## From (data=RDD, schema=StructType)

In [7]:
from pyspark.sql.types import (
    Row,
    StructField,
    StructType,
    StringType,
    IntegerType
)

rdd = spark.sparkContext.parallelize([
    {"name": "messi", "goal": 1}, 
    {"name": "ronald", "goal": 2}, 
    {"name": "messi", "goal": 1}, 
    {"name": "ronald", "goal": 4}
])
schema=StructType([
   StructField("name", StringType(), True),
   StructField("goal", IntegerType(), True)
])

df = spark.createDataFrame(data=rdd, schema=schema)
df.show()

+------+----+
|  name|goal|
+------+----+
| messi|   1|
|ronald|   2|
| messi|   1|
|ronald|   4|
+------+----+



## From (data=RDD)

In [8]:
rdd = spark.sparkContext.parallelize([
    ("messi", 1), 
    ("ronald", 2), 
    ("messi", 3), 
    ("ronald", 4)
])

Score = Row('name', 'goal')
scores = rdd.map(lambda row: Score(*row))
print(type(scores))

df = spark.createDataFrame(data=scores)
df.show()

<class 'pyspark.rdd.PipelinedRDD'>


+------+----+
|  name|goal|
+------+----+
| messi|   1|
|ronald|   2|
| messi|   3|
|ronald|   4|
+------+----+



# Cleanup

In [9]:
del spark
gc.collect()

289